In [13]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
from torch.autograd.functional import jacobian
from operators import Δ, div

In [10]:
var = torch.tensor([[1.0, 1.0],
                   [4.0, 3.0]], requires_grad=True)
t = torch.tensor([[2.0], 
                  [3.0]], requires_grad=True)

var_2 = torch.tensor([[1.0],
                   [3.0]], requires_grad=True)
t_2 = torch.tensor([[1.0], 
                  [2.0]], requires_grad=True)
def u(x, t):
    xt = torch.cat((x, t), dim=1)
    return (1/6)*torch.pow(xt[:, 0], 3) + xt[:,-1]

In [15]:
spatial_net = nn.Sequential(
            nn.Linear(2, 4),
            nn.ELU(),
            nn.Linear(4, 4),
            nn.ELU(),
            nn.Linear(4, 1)
        )

In [3]:
out = u(var, t)
out_2 = u(var_2, t_2)

In [22]:
torch.sum(jacobian(spatial_net, var), dim=0)

tensor([[[0.0238, 0.0103],
         [0.0376, 0.0108]]])

In [19]:
out = spatial_net(var)
div(out, var)

tensor([[0.0238, 0.0103],
        [0.0376, 0.0108]], grad_fn=<MmBackward>)

In [4]:
print(div(out.unsqueeze(1), t))
print(div(out_2.unsqueeze(1), t_2))

tensor([[1.],
        [1.]], grad_fn=<SliceBackward>)
tensor([[1.],
        [1.]], grad_fn=<SliceBackward>)


In [5]:
print(div(out.unsqueeze(1), var))
print(div(out_2.unsqueeze(1), var_2))

tensor([[0.5000],
        [8.0000]], grad_fn=<SliceBackward>)
tensor([[0.5000],
        [4.5000]], grad_fn=<SliceBackward>)


In [6]:
print(Δ(out.unsqueeze(1), var))
print(Δ(out_2.unsqueeze(1), var_2))

tensor([[1.],
        [4.]], grad_fn=<UnsqueezeBackward0>)
tensor([[1.],
        [3.]], grad_fn=<UnsqueezeBackward0>)


In [7]:
loss = torch.square(div(out, t) - Δ(out, var)).mean()

In [8]:
loss.backward()

<ipython-input-9-09a6243c8d35>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  loss.grad
